In [1]:
import init
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common import simple_sbml
from SBMLLint.common.reaction import Reaction
from SBMLLint.tools import sbmllint
from SBMLLint.tools import print_reactions

import os
import numpy as np
import pandas as pd
from pulp import *
import matplotlib.pyplot as plt
import time

from scipy.linalg import lu, inv
from scipy.optimize import linprog

In [2]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.simple_sbml import SimpleSBML
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix

from SBMLLint.games.som import SOM
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction

Current Directory: /Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebooks


In [3]:
RECON3D = "Recon3D.xml"

In [4]:
def analyzeByGAMES(model_file):
  simple = SimpleSBML()
  simple.initialize(os.path.join(cn.BIGG_DIR, model_file))
  print("Model: %s" % model_file)
  xgames_start = time.time()
  m = GAMES_PP(simple)
  res = m.analyze(simple_games=False, error_details=False, suppress_message=True)
  xgames_end = time.time()
  xgames_time = xgames_end - xgames_start
  print("xGAMES time: %f" % xgames_time)
  if res:
    print("xGAMES found an error!")
  else:
    print("xGAMES didn't find an error!")
  return m, xgames_time

def getISS(m_games):
  result = []
  gr = GAMESReport(m_games)
  operation_df = gr.getOperationMatrix()
  if m_games.echelon_errors:
    for reaction in m_games.echelon_errors:
      operation_series = operation_df.T[reaction.label]
      result.append(len(operation_series.to_numpy().nonzero()[0]))
  if m_games.type_three_errors:
    for reaction in m_games.echelon_errors:
      operation_series = operation_df.T[reaction.label]
      result.append(len(operation_series.to_numpy().nonzero()[0]))
  print(result)
  return(result)

In [5]:
games_10000, time_10000 = analyzeByGAMES(RECON3D)

Model: Recon3D.xml
xGAMES time: 24282.472383
xGAMES found an error!


In [6]:
m_10000_ris = getISS(games_10000)
np.mean(m_10000_ris)

[800, 733, 804, 791, 824, 862, 772, 810, 847, 824, 800, 830, 774, 817, 852, 814, 819, 831, 863, 806, 803, 782, 787, 786, 801, 811, 830, 824, 800, 814, 846, 863, 815, 812, 795, 865, 818, 782, 807, 800, 803, 799, 750, 772, 822, 819, 827, 847, 825, 3, 826, 3, 853, 814, 815, 839, 821, 800, 827, 638, 813, 807, 858, 818, 815, 847, 848, 851, 831, 830, 830, 788, 787, 802, 847, 815, 786, 815, 819, 855, 814, 726, 750, 819, 848, 788, 803, 787, 841, 839, 811, 801, 860, 803, 862, 802, 815, 816, 742, 809, 857, 794, 815, 801, 801, 807, 806, 816, 799, 814, 807, 803, 855, 815, 853, 863, 809, 853, 822, 767, 820, 818, 768, 850, 768, 815, 799, 821, 821, 801, 638, 810, 754, 783, 818, 811, 819, 812, 816, 806, 818, 789, 825, 794, 800, 810, 802, 811, 843, 804, 804, 808, 802, 810, 741, 609, 848, 864, 637, 793, 863, 789, 865, 767, 787, 786, 812, 746, 823, 811, 830, 811, 854, 851, 818, 820, 811, 818, 825, 813, 803, 856, 817, 827, 804, 830, 818, 848, 850, 846, 830, 859, 862, 811, 789, 863, 794, 862, 608, 812, 846

801.9023199023198

In [7]:
len(m_10000_ris)

1638

In [8]:
m2_games_10000, m2_time_10000 = analyzeByGAMES(RECON3D)

Model: Recon3D.xml
xGAMES time: 28342.741264
xGAMES didn't find an error!


In [9]:
m3_games_10000, m3_time_10000 = analyzeByGAMES(RECON3D)

Model: Recon3D.xml
xGAMES time: 23428.722166
xGAMES didn't find an error!
